In [1]:
%run __init__.py
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import open3d as o3d

In [3]:
pnp_results = np.load('../calib_output/pnp_results.npy', allow_pickle=True).item()

In [4]:
r_pointclouds = np.load('../calib_output/r_pcd.npy', allow_pickle=True).item()

In [11]:
for key in pnp_results.keys():
    print(pnp_results[key][0]) # PnP transformation T: chessboard -> camera
    print(pnp_results[key][1].shape) # Transformed keypoints from chessboard: T @ kp_chessboard
    print(r_pointclouds[key].shape) # Segmented chessboard from Azure
    break

[[ 0.99127092  0.00252416  0.13181648 -0.01185215]
 [ 0.00531094  0.99824066 -0.0590541  -0.10442502]
 [-0.13173363  0.05923868  0.98951353  1.28062433]
 [ 0.          0.          0.          1.        ]]
(48, 3)
(21730, 3)


In [ ]:
# pi = mrob.registration.estimate_plane(t_scene_points[0])
# print(pi)

In [ ]:
x = o3d.geometry.PointCloud()
x.points = o3d.utility.Vector3dVector(t_scene_points)
#o3d.visualization.draw_geometries([x])
print(len(pnp_results))

In [22]:
import mrob
#initialization
N = 100 # len(pnp_results)
graph = mrob.FGraph()
W = np.array([1])
n1 = graph.add_node_pose_3d(mrob.geometry.SE3())

In [ ]:
for n in range(N):
    #load PnP
    item = pnp_results[list(pnp_results.keys())[n]]
    Y = item[1] #point cloud
    normal = mrob.registration.estimate_normal(X)
    #Y_center = mrob.registration.estimate_centroid(X)
    #print(normal)
    
    #load pcd
    item = pnp_results[list(pnp_results.keys())[n]]
    X = item[1] #centroid of 
    N_points = Y.shape[0]
    
    #create factors
    for i in range(N_points):
        graph.add_factor_1pose_point2plane(z_point_x = X[i],
                                       z_point_y = Y[i], #Y_center if it is not working well
                                       z_normal_y = normal,
                                       nodePoseId = n1,
                                       obsInf = W)
# solve the problem
graph.solve(mrob.LM)
result = graph.get_estimated_state()
print(result[0])

In [7]:
# test visually
pcx = o3d.geometry.PointCloud()
pcx.points= o3d.utility.Vector3dVector(X)
pcy = o3d.geometry.PointCloud()
pcy.points= o3d.utility.Vector3dVector(Y)
pcx.transform(result[0])
o3d.visualization.draw_geometries([pcx,pcy])